## Inizio Quarta parte

### Carico il DF su Big Query

In [26]:
%run functions.ipynb

In [27]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = 'pipeline/final_df4.parquet'

# Initialize the GCS client
client = storage.Client()

# Load the Parquet files
final_df4 = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [28]:
final_df4.describe()

,date,sentiment,relevance,sharesFacebook,importanceRank,alexaGlobalRank,alexaCountryRank,date_column,year,month,...,WORK_OF_ART_mean_salience,CONSUMER_GOOD_mean_salience,NUMBER_mean_salience,DATE_mean_salience,PRICE,PRICE_mean_salience,ADDRESS,ADDRESS_mean_salience,PHONE_NUMBER,PHONE_NUMBER_mean_salience
count,4322,0.0,4322.0,4322.000000,4322.000000,4322.00000,4322.000000,4322,4322.0,4322.000000,...,4322.000000,4322.000000,4322.0,4322.0,4322.000000,4322.0,4322.000000,4322.0,4322.000000,4322.0
mean,2023-09-08 12:08:39.759370752,NaN,1.0,27.828089,217544.448172,17445.07404,514.228366,2023-09-08 12:08:39.759370752,2023.0,8.761684,...,0.008127,0.008544,0.0,0.0,0.333873,0.0,0.009024,0.0,0.006016,0.0
min,2023-01-01 00:00:00,NaN,1.0,0.000000,164339.000000,0.00000,0.000000,2023-01-01 00:00:00,2023.0,1.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
25%,2023-05-15 06:00:00,NaN,1.0,1.000000,170621.000000,1633.00000,30.000000,2023-05-15 06:00:00,2023.0,5.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
50%,2023-11-27 00:00:00,NaN,1.0,7.000000,191788.000000,3336.00000,47.000000,2023-11-27 00:00:00,2023.0,11.000000,...,0.001546,0.002038,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
75%,2023-12-16 00:00:00,NaN,1.0,26.000000,211051.000000,5616.00000,67.000000,2023-12-16 00:00:00,2023.0,12.000000,...,0.007560,0.007904,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
max,2023-12-21 00:00:00,NaN,1.0,2544.000000,1000000.000000,302718.00000,92187.000000,2023-12-21 00:00:00,2023.0,12.000000,...,0.314216,0.380333,0.0,0.0,30.000000,0.0,3.000000,0.0,5.000000,0.0
std,NaN,NaN,0.0,84.886091,97299.248453,38944.66327,5600.385508,NaN,0.0,4.027519,...,0.018242,0.019711,0.0,0.0,1.276532,0.0,0.112459,0.0,0.107402,0.0


In [29]:
final_df4.columns

Index(['uri', 'url', 'title', 'body', 'date', 'time', 'dateTime',
       'dateTimePub', 'lang', 'isDuplicate', 'dataType', 'sentiment',
       'eventUri', 'relevance', 'image', 'authors', 'sharesFacebook',
       'sourceTitle', 'sourceLocationLabel', 'categoryLabels',
       'categoryWeights', 'importanceRank', 'alexaGlobalRank',
       'alexaCountryRank', 'date_column', 'year', 'month', 'year_month',
       'topic', 'split', 'shares_scaled', 'body_pre', 'score', 'magnitude',
       'num_documents', 'Toxic', 'Insult', 'Profanity', 'Derogatory', 'Sexual',
       'Death_Harm__Tragedy', 'Violent', 'Firearms__Weapons', 'Public_Safety',
       'Health', 'Religion__Belief', 'Illicit_Drugs', 'War__Conflict',
       'Politics', 'Finance', 'Legal', 'PERSON', 'OTHER', 'ORGANIZATION',
       'EVENT', 'LOCATION', 'WORK_OF_ART', 'CONSUMER_GOOD', 'NUMBER', 'DATE',
       'PERSON_mean_salience', 'OTHER_mean_salience',
       'ORGANIZATION_mean_salience', 'EVENT_mean_salience',
       'LOCATION_mean_s

In [30]:
nan_count = final_df4.isna().sum()
print("Total NaN values:", nan_count)

Total NaN values: uri                           0
url                           0
title                         0
body                          0
date                          0
                             ..
PRICE_mean_salience           0
ADDRESS                       0
ADDRESS_mean_salience         0
PHONE_NUMBER                  0
PHONE_NUMBER_mean_salience    0
Length: 75, dtype: int64


In [31]:
print(final_df4['split'].value_counts())

split
TRAIN       2408
TEST         958
VALIDATE     956
Name: count, dtype: int64


In [32]:
final_df4 = final_df4.drop(columns="DATE")

In [33]:
final_df4.columns

Index(['uri', 'url', 'title', 'body', 'date', 'time', 'dateTime',
       'dateTimePub', 'lang', 'isDuplicate', 'dataType', 'sentiment',
       'eventUri', 'relevance', 'image', 'authors', 'sharesFacebook',
       'sourceTitle', 'sourceLocationLabel', 'categoryLabels',
       'categoryWeights', 'importanceRank', 'alexaGlobalRank',
       'alexaCountryRank', 'date_column', 'year', 'month', 'year_month',
       'topic', 'split', 'shares_scaled', 'body_pre', 'score', 'magnitude',
       'num_documents', 'Toxic', 'Insult', 'Profanity', 'Derogatory', 'Sexual',
       'Death_Harm__Tragedy', 'Violent', 'Firearms__Weapons', 'Public_Safety',
       'Health', 'Religion__Belief', 'Illicit_Drugs', 'War__Conflict',
       'Politics', 'Finance', 'Legal', 'PERSON', 'OTHER', 'ORGANIZATION',
       'EVENT', 'LOCATION', 'WORK_OF_ART', 'CONSUMER_GOOD', 'NUMBER',
       'PERSON_mean_salience', 'OTHER_mean_salience',
       'ORGANIZATION_mean_salience', 'EVENT_mean_salience',
       'LOCATION_mean_salience'

In [34]:
# Upload to BQ :        
ds_id = "datasetnlp"
project = !gcloud config get-value project
PROJECT_ID = project[0]
bucket = "ccai-storage"
table_id="finaldf5"
location = "europe-west3"  # Specify the location where your BigQuery dataset resides
upload_dataframe_to_bigquery(final_df4, table_id, PROJECT_ID, ds_id, bucket, location)

Loaded 4322 rows into BigQuery table: gcp-ccai-auto-ml-contactcenter.datasetnlp.stepfinalbq
